In [2]:
%%time
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, GridSearchCV

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.07 ms


In [3]:
##for printing multiple output..
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
train = pd.read_csv('../input/train_NIR5Yl1.csv')
test = pd.read_csv('../input/test_8i3B3FC.csv')

In [5]:
#Deleting outliers
train = train.drop(train[(train['Views']>3100000) | (train['Reputation'] > 900000) | (train['Upvotes'] > 210000) | (train['Answers'] > 65)].index)

In [6]:
train["Upvotes"] = np.log1p(train["Upvotes"])
train["Reputation"] = np.log1p(train["Reputation"])
train["Views"] = np.log1p(train["Views"])
train["Answers"] = np.log1p(train["Answers"])

test["Reputation"] = np.log1p(test["Reputation"])
test["Views"] = np.log1p(test["Views"])
test["Answers"] = np.log1p(test["Answers"])

In [8]:
train_labels = np.array(train['Upvotes'])
train_features = train.drop(['Upvotes','ID','Username'], axis=1)
train_features = pd.get_dummies(train_features)
train_features = train_features.fillna(0)

ID_test = test['ID']
test_features = test.drop(['ID','Username'], axis=1)
test_features = pd.get_dummies(test_features)

In [ ]:
##generate the cross validation fold.
st_train = train_features.values
st_test = test_features.values
Y = train_labels
#1052 (gamma = 0.9257,learning_rate = 0.2797,max_depth = 5,min_child_weight = 10,n_estimators = 305,subsample = 0.6239,colsample_bytree = 0.7443)
clf = xgb.XGBRegressor(gamma = 0.9257,learning_rate = 0.2797,max_depth = 5,min_child_weight = 9,n_estimators = 305,subsample = 0.6239,colsample_bytree = 0.7443)
fold = 5
cv = KFold(n_splits=fold, shuffle=True, random_state=2018)
X_preds = np.zeros(st_train.shape[0])
preds = np.zeros(st_test.shape[0])
for i, (tr, ts) in enumerate(cv.split(st_train)):
    print(ts.shape)
    mod = clf.fit(st_train[tr], Y[tr])
    X_preds[ts] = mod.predict(st_train[ts])
    preds += mod.predict(st_test)
    print("fold {}, RMSE : {:.3f}".format(i, sqrt(mean_squared_error(Y[ts], X_preds[ts]))))
score = sqrt(mean_squared_error(Y, X_preds))
print(score)
preds1 = preds/fold

In [ ]:
preds1 = np.abs(np.expm1(preds1))
sub = pd.DataFrame({'ID': ID_test, 'Upvotes': preds1})
sub=sub.reindex(columns=["ID","Upvotes"])
sub.to_csv('submission.csv', index=False)